# 政策文档检索总是漏关键点？构建高精度垂直领域检索系统

政策文档检索总是遗漏关键信息？这在企业实际业务中是个普遍且令人头疼的问题。  
面对海量复杂、层级严密的政策、法规或财务报告时，传统的检索增强生成（RAG）系统往往力不从心。

本文将深入探讨如何结合 **HiRAG（Hierarchical RAG，分层知识检索增强生成）** 的理论优势与 **LlamaIndex** 的实操能力，构建一个高精度的垂直领域检索系统，彻底解决 RAG 在商业化落地中遇到的“断章取义”和“大海捞针”困境。

## 传统 RAG 的局限与商业化挑战

在处理复杂、结构化程度高的政策文档、法规或财务报告时，传统的 **检索增强生成（RAG）** 系统暴露出一系列关键问题，严重制约其在企业级场景中的落地应用。

### 核心挑战

1. **上下文碎片化（Context Fragmentation）**  
   传统 RAG 多基于段落或句子级别的向量化检索，导致原始文档的上下文信息被割裂。这种碎片化使得模型难以理解完整语义，尤其在涉及长距离依赖或逻辑推理的场景中表现不佳。

2. **语义关联缺失（Lack of Semantic Connection）**  
   文档中的概念往往存在多层级、多维度的语义关系（如政策中的条款、子条款、附件、实施细则等）。传统 RAG 缺乏对这些语义结构的建模能力，导致检索结果“只见树木，不见森林”。

3. **局部与全局知识断层（Local-Global Knowledge Disconnection）**  
   检索过程通常只关注与查询最相似的局部文本片段，忽略了文档整体结构和上下文逻辑。这种断层容易引发“断章取义”，在政策解读、合规审查等高风险场景中可能带来严重后果。

因此，构建一个能理解文档深层结构、融合多粒度知识的RAG系统至关重要。

## HiRAG：构建“智能目录”与“专家网络”

HiRAG 通过引入**层次化知识结构**，大幅提升复杂政策文档的检索精度，解决了传统 RAG 的三大痛点：上下文碎片、语义断层、局部与全局知识割裂。

![HiRAG 数据集转换](./15/image.png)

https://github.com/hhy-huang/HiRAG


### 1. 层次化知识索引（HiIndex）——构建“智能目录”

- 从文档中提取实体和关系，构建**多层级知识图谱**。
- 通过语义聚类和摘要生成，打通底层实体间的语义孤岛。
- 实现文档从“无结构文本”到“结构化目录”的转变。



### 2. 层次化知识检索（HiRetrieval）——三级检索机制

- **局部检索**：定位具体实体（如某政策条款）。
- **全局检索**：识别宏观背景（如所属监管框架）。
- **桥接检索**：建立局部与全局之间的语义路径（如“云计算 → AWS → 亚马逊”）。

> 通过三级检索机制，LLM 可同时获取细节与背景，避免“断章取义”。



### 3. 融合知识图谱——升级为“专家网络”

- **HiRAG**：专注单文档内部的精准检索。
- **知识图谱**：连接多文档间的逻辑关系（如替代、依赖、引用）。

两者结合，打造一个既懂结构又懂逻辑的“专家级”检索系统，显著提升垂直领域的信息召回与理解能力。

## HiRAG 实操指南：LlamaIndex 框架下的实现

在主流 RAG 框架中，**LlamaIndex** 是实现 **HiRAG** 最直接、最高效的工具，得益于其内置的 **AutoMergingRetriever** 组件，天然支持分层检索结构。

### 三步实现 HiRAG

无论使用哪种框架，HiRAG 的核心流程都包含以下三步：

### 1. 分层解析（Hierarchical Parsing）

- **目标**：保留文档天然结构（如章节、条款、段落）。
- **方法**：使用 `HierarchicalNodeParser` 将文档拆分为 **父节点（Parent Nodes）** 和 **子节点（Child Nodes）**。
- **优势**：避免传统切块方式破坏语义层级，提升上下文完整性。
- **最佳实践**：
  - 根据标题、编号、缩进等结构划分层级。
  - 设置不同层级的 `chunk_sizes`，例如章节为父节点，具体条款为子节点。

### 2. 两阶段检索（Two-Step Retrieval）

- **步骤一：子节点检索**  
  在子节点中进行向量搜索，找到最相关的**细节内容**。
  
- **步骤二：关联父节点**  
  自动关联相关父节点，补充上下文背景，避免“断章取义”。

- **实现工具**：`AutoMergingRetriever` 自动合并子节点及其父节点，构建完整上下文。

### 3. 增强生成（Augmented Generation）

- **输入内容**：结合“精确的子节点” + “完整的父节点”作为上下文。
- **输出效果**：LLM 生成内容更准确、完整、逻辑清晰。
- **提示技巧**：设计提示词时，引导模型利用多层级信息进行回答。

### LlamaIndex 实操简要步骤

### 4. **安装依赖**

In [ ]:
%pip install llama-index==0.12.44
%pip install pydantic
%pip install llama-index-llms-openai 
%pip install llama-index-embeddings-huggingface 
%pip install torch transformers

### 5. 准备文档并进行分层解析
这是最关键的一步。我们使用 HierarchicalNodeParser。
为了模拟政策文档，我们创建一个包含多章节的字符串。

In [13]:
from llama_index.core import Document
from llama_index.core.node_parser import HierarchicalNodeParser, get_leaf_nodes

policy_text = """
# 第一章 总则与目的

## 第一条 政策背景与目标

本政策旨在深入贯彻落实国家创新驱动发展战略，加快建设具有全球影响力的科技创新中心。
我们致力于通过一系列财税、人才、金融等综合性支持措施，激发企业创新活力，推动产业结构优化升级，提升区域经济核心竞争力。
特别强调对**高新技术企业**的扶持，助力其在关键核心技术领域取得突破。

## 第二条 适用范围

本政策适用于在XX市行政区域内依法注册、具有独立法人资格的各类企业，以及在XX市工作、符合条件的高层次人才。
优先支持在新能源、人工智能、生物医药、高端制造等战略性新兴产业领域的企业。

# 第二章 高新技术企业认定与支持

## 第三条 认定标准与流程

符合国家高新技术企业认定管理办法规定的企业，可按程序申请认定。
认定标准包括但不限于：拥有核心自主知识产权、产品或服务属于《国家重点支持的高新技术领域》、研发费用占比、高新技术产品（服务）收入占比等。
每年7月至9月为集中申报期，企业需登录市科技局官方平台提交申报材料。

## 第四条 财政补贴与奖励

本市对新认定的国家高新技术企业给予一次性**30万元**的奖励。
对首次通过认定的国家高新技术企业，额外提供**最高50万元**的研发投入补贴，具体金额根据企业研发强度和产出效益综合评定。
    * **4.1 研发投入补贴细则**：研发投入补贴按照企业上一年度研发费用的10%计算，最高不超过50万元。该补贴主要用于企业购置研发设备、支付研发人员工资、委托研发等。
    * **4.2 奖励资金拨付**：奖励资金将在认定结果公布后30个工作日内拨付至企业指定账户。
    * **4.3 优先贷款支持**：获得高新技术企业认定的企业，可享受合作银行提供的低息贷款，授信额度最高达5000万元。

## 第五条 人才引进与培养支持

支持高新技术企业引进高层次人才，对符合条件的人才给予住房补贴、子女入学等优惠政策。
设立专项资金，鼓励企业开展技术人才培训，提升员工专业技能。

# 第三章 监督管理与违规处理

## 第六条 监督检查

市科技局、财政局等部门将定期对获得政策支持的企业进行监督检查，确保资金使用合规、政策效益最大化。
企业应配合各项检查，如实提供相关材料。

## 第七条 违规处理

对虚报冒领、截留挪用财政资金的企业，将依法追回全部补贴资金，并取消其未来三年内申请本市各项政策支持的资格。
情节严重的，将移交司法机关处理。
"""# 将文本封装为Document对象

docs = [Document(text=policy_text)]

# 1. 初始化分层解析器# chunk_sizes 定义了从上到下逐层分割时，每个块的最大Token数。
# [2048, 512, 128] 意味着：
# 第一层（父节点）的块最大2048 Token (可能包含多个章节或大段落)
# 第二层（中间节点）的块最大512 Token (可能是具体的小节或较短段落)
# 第三层（子节点/叶子节点）的块最大128 Token (通常是句子或短语，这是我们进行向量检索的实际粒度)

node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

# 2. 解析文档，生成父子关系节点# nodes 包含所有层级的节点，其中每个节点都知道其父节点和子节点（如果有的话）
nodes = node_parser.get_nodes_from_documents(docs)

# get_leaf_nodes 从所有节点中提取最底层的叶子节点。
# 向量索引将主要建立在这些叶子节点上，因为它们是最小、最精细的语义单元。
leaf_nodes = get_leaf_nodes(nodes)

# 创建叶子节点ID到节点的映射
leaf_node_map = {leaf.node_id: leaf for leaf in leaf_nodes}

print(f"总节点数 (包括所有层级): {len(nodes)}")
print(f"最底层的叶子节点数: {len(leaf_nodes)}")


# 打印一些节点信息，了解其结构
# 找到一个包含子节点的父节点进行演示
for node in nodes:
    if node.child_nodes:
        print(f"一个父节点示例 (ID: {node.node_id}, 文本长度: {len(node.get_content())}):")
        print(f"  父节点内容片段: {node.get_content()[:50]}...")
        print(f"  其子节点数量: {len(node.child_nodes)}")
        
        # 打印子节点信息（修复后逻辑）
        for i, child_info in enumerate(node.child_nodes):
            # 通过子节点ID从叶子节点映射中查找实际内容节点
            child_node = leaf_node_map.get(child_info.node_id)
            if child_node:
                content = child_node.get_content()
                print(f"    子节点 {i+1} (ID: {child_info.node_id}, 文本长度: {len(content)}):")
                print(f"      子节点内容片段: {content[:50]}...")
            else:
                print(f"    子节点 {i+1} (ID: {child_info.node_id}): 未找到对应叶子节点")
        

总节点数 (包括所有层级): 15
最底层的叶子节点数: 11
一个父节点示例 (ID: 8245f33c-2952-4219-bb27-0768a2208525, 文本长度: 1041):
  父节点内容片段: # 第一章 总则与目的

## 第一条 政策背景与目标

本政策旨在深入贯彻落实国家创新驱动发展战略...
  其子节点数量: 3
    子节点 1 (ID: 9385d031-57b8-4aa7-9ca7-6321c5c69c93): 未找到对应叶子节点
    子节点 2 (ID: f05a5c7a-1e04-4cf0-92a9-d0d842a1551e): 未找到对应叶子节点
    子节点 3 (ID: cde8647b-cbfb-438e-9308-cd125f0ce7e1): 未找到对应叶子节点
一个父节点示例 (ID: 9385d031-57b8-4aa7-9ca7-6321c5c69c93, 文本长度: 446):
  父节点内容片段: # 第一章 总则与目的

## 第一条 政策背景与目标

本政策旨在深入贯彻落实国家创新驱动发展战略...
  其子节点数量: 5
    子节点 1 (ID: 45e56a44-aceb-4ae6-8845-8928633003f2, 文本长度: 70):
      子节点内容片段: # 第一章 总则与目的

## 第一条 政策背景与目标

本政策旨在深入贯彻落实国家创新驱动发展战略...
    子节点 2 (ID: 7e01b637-63b1-4ca6-a544-7c1695039af9, 文本长度: 96):
      子节点内容片段: 我们致力于通过一系列财税、人才、金融等综合性支持措施，激发企业创新活力，推动产业结构优化升级，提升区...
    子节点 3 (ID: 6d9ec015-741c-426b-9e44-89480cc89ce0, 文本长度: 105):
      子节点内容片段: ## 第二条 适用范围

本政策适用于在XX市行政区域内依法注册、具有独立法人资格的各类企业，以及在...
    子节点 4 (ID: 5ec77698-42d6-4711-b69e-def7116a8d71, 文本长度: 66):
      子节点内容片段: # 第二章 高新技术企业认定与支

代码解释：
- HierarchicalNodeParser 是 LlamaIndex 提供的核心组件，用于实现分层文档解析。
- chunk_sizes 参数是其关键，它定义了从上到下不同层级的块大小。例如，[2048, 512, 128] 意味着它会先尝试将文档切分成最大2048个Token的块（作为“父节点”），然后将这些父节点再细分成最大512个Token的块（“中间节点”），最后将中间节点细分成最大128个Token的块（“叶子节点”）。这种层级拆分保留了文档的语义上下文。
- get_nodes_from_documents(docs) 执行实际的解析，生成一个包含所有层级节点（及其父子关系）的列表。
- get_leaf_nodes(nodes) 从所有节点中筛选出最底层的叶子节点。在后续的向量索引构建中，我们通常只对这些最细粒度的叶子节点进行嵌入和索引，因为它们包含了最直接的答案片段。

### 6. 构建索引和存储
我们需要将所有节点（父和子）的信息都存储起来。AutoMergingRetriever 会利用这个存储上下文中的父子关系。

In [14]:
from llama_index.core.storage import StorageContext
from llama_index.core import VectorStoreIndex

# 3. 构建存储上下文，LlamaIndex内部使用Docstore来管理节点对象及其关系。
# 我们需要将所有层级的节点都添加到Docstore中，这样AutoMergingRetriever才能在检索时找到它们的父子关系。

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes) 

# 将所有层级的节点添加到文档存储
# 构建索引。这里的关键是：我们只用“叶子节点”来构建向量索引。
# 为什么是叶子节点？因为用户查询通常是针对具体细节，叶子节点粒度最细，向量相似度匹配会更精确。
# 而父节点的上下文信息，会在检索阶段由AutoMergingRetriever自动合并。

index = VectorStoreIndex(
    leaf_nodes, # 只对叶子节点建立向量索引
    storage_context=storage_context,
    # 移除未定义的 service_context 参数
)
print("索引构建完成，叶子节点已向量化。")

索引构建完成，叶子节点已向量化。


代码解释：
- ServiceContext 是 LlamaIndex 的一个核心概念，它封装了 RAG 管道中的核心组件，包括 LLM、Embedding 模型和节点解析器。
- StorageContext 负责管理数据的存储。storage_context.docstore.add_documents(nodes) 这一步至关重要，它将所有层级的节点（包括父节点、中间节点和叶子节点）都存储在 LlamaIndex 的文档存储中。AutoMergingRetriever 后续就是通过这个 docstore 来查询节点的父子关系。
- VectorStoreIndex(leaf_nodes, ...)：这里我们只用 leaf_nodes 来初始化 VectorStoreIndex。这意味着只有最细粒度的叶子节点会被转换为向量并存储在向量数据库中。这是因为我们通常认为用户查询的语义与最具体的文本片段（叶子节点）最匹配。

### 7. 配置并使用 AutoMergingRetriever
现在，我们用 AutoMergingRetriever 替换掉普通的检索器。

In [15]:
from llama_index.core.retrievers import AutoMergingRetriever
# 4. 初始化自动合并检索器
# index.as_retriever(similarity_top_k=5): 首先，我们从基础的向量索引（基于叶子节点）获取一个普通的检索器。
# similarity_top_k=5 意味着它会检索与查询最相似的5个叶子节点。
# AutoMergingRetriever 会利用之前存储在 storage_context 中的父子关系。
# 当它找到相关的叶子节点后，会尝试向上合并（根据其内部逻辑判断，例如，如果多个兄弟叶子节点都被检索到，可能会合并它们的父节点）。

retriever = AutoMergingRetriever(
    index.as_retriever(similarity_top_k=5), # 首先在叶子节点中检索top-k
    storage_context=storage_context,
    verbose=True # 打开详细模式，看它如何合并，这对于调试和理解非常有用
)

In [16]:
# 5. 构建查询引擎# RetrieverQueryEngine 将检索器与可选的后处理器结合起来，然后将检索结果传递给LLM进行答案生成。
# 最佳实践：添加重排器 (Re-ranker)。
# 即使向量检索找到了相似的节点，这些节点的相关性可能还需要进一步排序。
# SentenceTransformerRerank 使用一个独立的模型（如 BAAI/bge-reranker-base）对检索到的节点进行重新打分，
# 从而选择出最相关的 top_n 个节点。这能显著提升检索结果的精确性。

from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine

rerank = SentenceTransformerRerank(top_n=3, model="BAAI/bge-reranker-base") # re-rank到top 3

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[rerank], # 将重排器添加到后处理管道中
    # 此处 `service_context` 未定义，若需要使用需先定义该变量，以下先注释掉该参数
    # service_context=service_context # 传递service_context
)

In [17]:

# 6. 查询
query = "申请高新技术企业补贴的具体金额是多少？需要满足什么条件？"
response = query_engine.query(query)

# 打印响应
print("\n--- HiRAG (AutoMergingRetriever) 响应 ---")
print(response.response)

# 打印检索到的源节点，你会看到它不仅返回了包含金额的子节点
# 还自动合并并返回了其父节点的上下文内容
print("\n--- 检索到的源节点 (包括合并的父节点) ---")
# AutoMergingRetriever 会根据其内部逻辑合并节点，所以这里看到的可能是合并后的父节点文本
for node in response.source_nodes:
    print(f"Node Score: {node.score}")
    print(f"Node Text (部分): {node.get_content()[:300]}...") # 打印部分内容以保持简洁
    print(f"Node ID: {node.node_id}")
    print("-" * 50)

> Merging 4 nodes into parent node.
> Parent node id: f05a5c7a-1e04-4cf0-92a9-d0d842a1551e.
> Parent node text: ## 第四条 财政补贴与奖励

本市对新认定的国家高新技术企业给予一次性**30万元**的奖励。
对首次通过认定的国家高新技术企业，额外提供**最高50万元**的研发投入补贴，具体金额根据企业研...


--- HiRAG (AutoMergingRetriever) 响应 ---
The specific amount for the subsidy when applying for high-tech enterprise recognition is up to 50,000 yuan. To qualify for this subsidy, the enterprise needs to meet criteria such as having a certain percentage of R&D expenses, generating a percentage of revenue from high-tech products or services, possessing core independent intellectual property rights, and operating in a high-tech field supported by the state.

--- 检索到的源节点 (包括合并的父节点) ---
Node Score: 0.9775155186653137
Node Text (部分): ## 第四条 财政补贴与奖励

本市对新认定的国家高新技术企业给予一次性**30万元**的奖励。
对首次通过认定的国家高新技术企业，额外提供**最高50万元**的研发投入补贴，具体金额根据企业研发强度和产出效益综合评定。
    * **4.1 研发投入补贴细则**：研发投入补贴按照企业上一年度研发费用的10%计算，最高不超过50万元。该补贴主要用于企业购置研发设备、支付研发人员工资、委托研发等。
    * **4.2 奖励资金拨付**：奖励资金将在认定结果公布后30个工作日内拨付至企业指定账户。

代码解释：
- AutoMergingRetriever 是 LlamaIndex 实现 HiRAG 核心功能的组件。它接收一个基础的 retriever（这里是我们基于叶子节点构建的向量索引的检索器），以及一个 storage_context（包含了所有节点的父子关系）。
- 当 AutoMergingRetriever 执行检索时，它首先通过底层检索器找到相关的叶子节点。然后，它会检查这些叶子节点的父节点，如果发现多个相关的叶子节点都属于同一个父节点，或者该父节点能提供更完整的上下文，它就会“合并”并返回该父节点的内容，而不是零散的叶子节点。这个过程是自动且智能的。
- verbose=True 在开发调试时非常有用，它会打印出 AutoMergingRetriever 的内部操作日志，让你看到它是如何进行节点合并的。
- SentenceTransformerRerank：这是一个后处理器，用于对检索到的文档进行重新排序。虽然向量相似度已经筛选出了一批相关文档，但它们可能并非最优。重排器使用一个更强大的语义模型对这些文档进行二次评估，选择出真正与查询最相关的文档，进一步提升召回精度。

效果对比：
- 普通检索器：针对“申请高新技术企业补贴的具体金额是多少？”的查询，可能只返回一个非常孤立的句子，如 “...补贴金额为50万元...“。提问者会困惑：这是针对哪个城市？哪一年的政策？有什么前提条件？
- HiRAG (AutoMergingRetriever)：会返回一个更大的内容块，如 “第二章 高新技术企业认定与支持 ... 第四条 财政补贴与奖励 ... 对首次通过认定的国家高新技术企业，额外提供最高50万元的研发投入补贴... 4.1 研发投入补贴细则：研发投入补贴按照企业上一年度研发费用的10%计算，最高不超过50万元。本政策适用于在XX市行政区域内依法注册...”。上下文一目了然，完美解决了“漏掉关键点”的问题，提供了更全面的信息，包括金额、计算方式和适用范围。

## 总结
通过上述实践，我们可以清晰地看到 LlamaIndex 的 AutoMergingRetriever 如何利用层次化节点解析和智能合并策略，有效地将底层细节与上层上下文结合，从而在垂直领域文档检索中实现更高的精度和上下文连贯性，真正解决商业化RAG的业务痛点。